In [139]:
import pandas as pd

In [140]:
data=pd.read_csv("final_data.csv")

In [141]:
data.head()

,key,appln_id,mainarea34,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,similar_patents,similar_patents_familyIDs,num_unique_familyIDs
0,EP1321051A2,19,Other Fields,19188314.0,286949845.0,2001-12-21,EP,Y,EP,1321051,A2,"['US5759163A', 'WO9410865A1', 'EP0664965A1', '...","['25279710', '25522562', '22699396', '22758306...",1854
1,EP1323868A1,20,Other Fields,19774416.0,287579451.0,2001-12-20,EP,Y,EP,1323868,A1,"['EP0226251A1', 'EP0672795A1', 'WO8002359A1', ...","['19846985', '19863945', '6070111', '27544489'...",1899
2,EP1240845A1,23,Other Fields,8861213.0,289129312.0,2001-03-16,EP,Y,EP,1240845,A1,"['WO0187245A3', 'WO0187245A2', 'US2002010556A1...","['27075299', '27075299', '24279046', '26914213...",1620
3,EP1452428A1,33,Mechanical engineering,19184257.0,287898353.0,2001-12-10,EP,Y,EP,1452428,A1,"['US2004079004A1', 'WO03049990A1', 'EP1304282A...","['19184257', '19184257', '19140954', '18907837...",1837
4,EP1733650A1,76,Other Fields,35589510.0,279068442.0,2005-06-15,EP,Y,EP,1733650,A1,"['WO9603606A1', 'EP0181477A2', 'WO9321868A1', ...","['3515062', '6247500', '3501175', '25076445', ...",1685


In [142]:
import os
from dotenv import load_dotenv

In [143]:
import mysql.connector

In [144]:
load_dotenv()
db_config = {
    'host': os.getenv('DB_HOST'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'database': os.getenv('DB_DATABASE')
}

In [145]:
data['docdb_family_id']

0         19188314.0
1         19774416.0
2          8861213.0
3         19184257.0
4         35589510.0
             ...    
124101    18225474.0
124102    18500622.0
124103    12004924.0
124104    15170543.0
124105    17518030.0
Name: docdb_family_id, Length: 124106, dtype: float64

In [146]:
conn = mysql.connector.connect(**db_config)
family_ids=tuple(data['docdb_family_id'])
cursor = conn.cursor(dictionary=True)
query = f"""
SELECT    
	docdb_family_id,
    JSON_ARRAYAGG(cited_docdb_family_id) AS cited_docdb_family_id
FROM
    patstat_s24.tls228_docdb_fam_citn
WHERE
    docdb_family_id IN {family_ids}
GROUP BY 
    docdb_family_id;
"""
cursor.execute(query)
results = cursor.fetchall()
cursor.close()

True

In [147]:
table2=pd.DataFrame(results)

In [148]:
table2.head()

,docdb_family_id,cited_docdb_family_id
0,1239749,"[6245732, 6270984, 6282047, 6313094, 6323632, ..."
1,3411930,"[13183030, 14441561, 14972801, 15124933, 15899..."
2,3460833,"[9116146, 23355200]"
3,3460835,"[22189292, 23471917, 24374969, 25442511]"
4,3460912,"[8274171, 20166258, 22433470, 22661073, 227366..."


In [149]:
len(table2)

123317

In [150]:
data.head()

,key,appln_id,mainarea34,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,similar_patents,similar_patents_familyIDs,num_unique_familyIDs
0,EP1321051A2,19,Other Fields,19188314.0,286949845.0,2001-12-21,EP,Y,EP,1321051,A2,"['US5759163A', 'WO9410865A1', 'EP0664965A1', '...","['25279710', '25522562', '22699396', '22758306...",1854
1,EP1323868A1,20,Other Fields,19774416.0,287579451.0,2001-12-20,EP,Y,EP,1323868,A1,"['EP0226251A1', 'EP0672795A1', 'WO8002359A1', ...","['19846985', '19863945', '6070111', '27544489'...",1899
2,EP1240845A1,23,Other Fields,8861213.0,289129312.0,2001-03-16,EP,Y,EP,1240845,A1,"['WO0187245A3', 'WO0187245A2', 'US2002010556A1...","['27075299', '27075299', '24279046', '26914213...",1620
3,EP1452428A1,33,Mechanical engineering,19184257.0,287898353.0,2001-12-10,EP,Y,EP,1452428,A1,"['US2004079004A1', 'WO03049990A1', 'EP1304282A...","['19184257', '19184257', '19140954', '18907837...",1837
4,EP1733650A1,76,Other Fields,35589510.0,279068442.0,2005-06-15,EP,Y,EP,1733650,A1,"['WO9603606A1', 'EP0181477A2', 'WO9321868A1', ...","['3515062', '6247500', '3501175', '25076445', ...",1685


In [151]:
len(data)

124106

In [152]:
124106-123317

789

In [153]:
#789 families for which citations were not found

In [155]:
data_with_citations = data.merge(table2, on='docdb_family_id', how='inner')

In [156]:
data_with_citations.head()

,key,appln_id,mainarea34,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,similar_patents,similar_patents_familyIDs,num_unique_familyIDs,cited_docdb_family_id
0,EP1321051A2,19,Other Fields,19188314.0,286949845.0,2001-12-21,EP,Y,EP,1321051,A2,"['US5759163A', 'WO9410865A1', 'EP0664965A1', '...","['25279710', '25522562', '22699396', '22758306...",1854,"[14362760, 14548413, 15480922, 17158120, 18647..."
1,EP1323868A1,20,Other Fields,19774416.0,287579451.0,2001-12-20,EP,Y,EP,1323868,A1,"['EP0226251A1', 'EP0672795A1', 'WO8002359A1', ...","['19846985', '19863945', '6070111', '27544489'...",1899,"[12053764, 14053006, 16624311, 19863945]"
2,EP1240845A1,23,Other Fields,8861213.0,289129312.0,2001-03-16,EP,Y,EP,1240845,A1,"['WO0187245A3', 'WO0187245A2', 'US2002010556A1...","['27075299', '27075299', '24279046', '26914213...",1620,"[7880649, 9350696, 9492399, 21776916, 21828725..."
3,EP1452428A1,33,Mechanical engineering,19184257.0,287898353.0,2001-12-10,EP,Y,EP,1452428,A1,"['US2004079004A1', 'WO03049990A1', 'EP1304282A...","['19184257', '19184257', '19140954', '18907837...",1837,"[3400054, 11582783, 12442126, 12462676, 142157..."
4,EP1733650A1,76,Other Fields,35589510.0,279068442.0,2005-06-15,EP,Y,EP,1733650,A1,"['WO9603606A1', 'EP0181477A2', 'WO9321868A1', ...","['3515062', '6247500', '3501175', '25076445', ...",1685,"[5930934, 10786628, 10798844, 10839944, 186096..."


In [162]:
data_with_citations.to_csv("data_with_citations.csv",index=False)

In [157]:
len(data_with_citations)

123317

In [158]:
result_df = data.merge(table2, on='docdb_family_id', how='left', indicator=True)
result_df.head()

,key,appln_id,mainarea34,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,similar_patents,similar_patents_familyIDs,num_unique_familyIDs,cited_docdb_family_id,_merge
0,EP1321051A2,19,Other Fields,19188314.0,286949845.0,2001-12-21,EP,Y,EP,1321051,A2,"['US5759163A', 'WO9410865A1', 'EP0664965A1', '...","['25279710', '25522562', '22699396', '22758306...",1854,"[14362760, 14548413, 15480922, 17158120, 18647...",both
1,EP1323868A1,20,Other Fields,19774416.0,287579451.0,2001-12-20,EP,Y,EP,1323868,A1,"['EP0226251A1', 'EP0672795A1', 'WO8002359A1', ...","['19846985', '19863945', '6070111', '27544489'...",1899,"[12053764, 14053006, 16624311, 19863945]",both
2,EP1240845A1,23,Other Fields,8861213.0,289129312.0,2001-03-16,EP,Y,EP,1240845,A1,"['WO0187245A3', 'WO0187245A2', 'US2002010556A1...","['27075299', '27075299', '24279046', '26914213...",1620,"[7880649, 9350696, 9492399, 21776916, 21828725...",both
3,EP1452428A1,33,Mechanical engineering,19184257.0,287898353.0,2001-12-10,EP,Y,EP,1452428,A1,"['US2004079004A1', 'WO03049990A1', 'EP1304282A...","['19184257', '19184257', '19140954', '18907837...",1837,"[3400054, 11582783, 12442126, 12462676, 142157...",both
4,EP1733650A1,76,Other Fields,35589510.0,279068442.0,2005-06-15,EP,Y,EP,1733650,A1,"['WO9603606A1', 'EP0181477A2', 'WO9321868A1', ...","['3515062', '6247500', '3501175', '25076445', ...",1685,"[5930934, 10786628, 10798844, 10839944, 186096...",both


In [159]:
data_no_citations=result_df[result_df['_merge']=="left_only"]

In [172]:
data_no_citations.drop(columns='_merge',inplace=True)

/tmp/ipykernel_31040/3604542973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_citations.drop(columns='_merge',inplace=True)


In [173]:
data_no_citations.head()

,key,appln_id,mainarea34,docdb_family_id,pat_publn_id,earliest_filing_date,appln_auth,granted,publn_auth,publn_nr,publn_kind,similar_patents,similar_patents_familyIDs,num_unique_familyIDs,cited_docdb_family_id
18,EP2000606A2,221,Other Fields,38513906.0,55346442.0,2007-06-05,EP,N,EP,2000606,A2,"['US2008115442A1', 'WO2004067871A1', 'WO016900...","['29721397', '29721397', '9887732', '6820748',...",1582,NaN
173,EP2116392A2,3008,Other Fields,40303316.0,272999459.0,2008-05-06,EP,N,EP,2116392,A2,"['EP0540390A1', 'EP0723946A1', 'EP0437662A1', ...","['9418402', '8002620', '8203512', '8871134', '...",1528,NaN
182,EP2002763A2,3083,Other Fields,38319065.0,275571493.0,2007-06-11,EP,N,EP,2002763,A2,"['EP0302191A1', 'EP1120356A1', 'EP0269757A1', ...","['22019756', '8174092', '8195628', '24520931',...",1660,NaN
198,EP2003277A2,3216,Other Fields,39832251.0,275591766.0,2007-02-28,EP,N,EP,2003277,A2,"['US5250765A', 'US2006137136A1', 'US2008178422...","['15604072', '36609685', '36609685', '34269607...",1509,NaN
219,EP2009198A2,3607,Other Fields,39791540.0,55535689.0,2007-06-25,EP,N,EP,2009198,A2,"['US2006157302A1', 'WO2004025049A1', 'WO870697...","['31987574', '31987574', '25334975', '36685611...",1617,NaN


In [174]:
len(data_no_citations)

789

In [175]:
data_no_citations.to_csv("data_no_citations.csv",index=False)

In [ ]:
#data with citations stored in data_with_citations.csv
#789 families with no citations stored in data_no_citations.csv
#query- should we store the unique similar family ids and unique cited family ids in a different column

In [176]:
123317+789

124106

In [177]:
data.columns

Index(['key', 'appln_id', 'mainarea34', 'docdb_family_id', 'pat_publn_id',
       'earliest_filing_date', 'appln_auth', 'granted', 'publn_auth',
       'publn_nr', 'publn_kind', 'similar_patents',
       'similar_patents_familyIDs', 'num_unique_familyIDs'],
      dtype='object')

In [178]:
len(data.columns)

14

In [179]:
data_no_citations.columns

Index(['key', 'appln_id', 'mainarea34', 'docdb_family_id', 'pat_publn_id',
       'earliest_filing_date', 'appln_auth', 'granted', 'publn_auth',
       'publn_nr', 'publn_kind', 'similar_patents',
       'similar_patents_familyIDs', 'num_unique_familyIDs',
       'cited_docdb_family_id'],
      dtype='object')

In [180]:
len(data_no_citations.columns)

15

In [181]:
data_with_citations.columns

Index(['key', 'appln_id', 'mainarea34', 'docdb_family_id', 'pat_publn_id',
       'earliest_filing_date', 'appln_auth', 'granted', 'publn_auth',
       'publn_nr', 'publn_kind', 'similar_patents',
       'similar_patents_familyIDs', 'num_unique_familyIDs',
       'cited_docdb_family_id'],
      dtype='object')

In [182]:
len(data_with_citations.columns)

15

In [1]:
import pandas as pd

In [3]:
dd=pd.read_csv("data_with_citations.csv")

In [ ]:
dd.head()